In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import sklearn

In [37]:
meme = pd.read_csv('drive/My Drive/memeproject/Danila FFmemes/danila_meme.csv')
event = pd.read_csv('drive/My Drive/memeproject/Danila FFmemes/danila_event.csv')
user = pd.read_csv('drive/My Drive/memeproject/Danila FFmemes/danila_user.csv')

# Описания колонок
CASE *language_id* WHEN 1 THEN 'ww' WHEN 2 THEN 'ru' WHEN 3 THEN 'en' ELSE null


# Фичи

**Избавляемся от сломанных мемов**

In [38]:
meme = meme.loc[meme.is_broken==False]

**Сумма лайков паблика**

In [39]:
source_like = meme[['source_id','likes']].groupby('source_id').sum()
source_like.head()

,likes
source_id,
1.0,141
2.0,8753
3.0,17852
4.0,17265
5.0,11334


**Сумма дизлайков паблика**

In [40]:
source_dislike = meme[['source_id','dislikes']].groupby('source_id').sum()
source_dislike.head()

,dislikes
source_id,
1.0,447
2.0,11876
3.0,19303
4.0,27841
5.0,15323


**Посчитаем количество лайков пары юзер-источник**

Сопоставим каждому мему его источник

In [41]:
event = pd.merge(event, meme[['meme_id','source_id']], on='meme_id')

Таблица лайков юзер-источник

In [42]:
event.loc[event.reaction_id == 3, 'reaction_id'] = 1
user_source_like = event.loc[event.reaction_id == 1][['user_id', 'source_id','reaction_id']].groupby(['user_id', 'source_id']).count()
user_source_like.head()

reaction_id
user_id source_id             
1615    8.0                  1
        11.0                 1
        13.0                 2
        18.0                 1
        25.0                 1

Таблица дизлайков юзер-источник

In [43]:
event.loc[event.reaction_id == 4, 'reaction_id'] = 2
user_source_dislike = event.loc[event.reaction_id == 2][['user_id', 'source_id','reaction_id']].groupby(['user_id', 'source_id']).count()
user_source_dislike.head()

reaction_id
user_id source_id             
1615    3.0                  2
        15.0                 2
        20.0                 2
        27.0                 1
        30.0                 1

# План по рек системе

## Постановка задачи

На вход дается пара (мем, юзер). Задача бинарной классификации: на выход нужно предсказать, понравится (1) ли мем юзеру или нет (0). Можно предсказать вероятность.

## Данные

Есть таблички events, memes, user-source (тут суперлайк считается за 5 лайков, а удалить считается за 5 дизлайков).
Можно делать какую угодно рекомендательную систему.

## Анализ данных

Список интересных вопросов:

• В какой момент юзеры отваливаются
• Отличается ли поведение англ/ру юзеров, статистика англ/ру мемов


Ниже "классический" вариант фич, не учитывающий в явном виде user_id и meme_id:

• сумма лайков мема
• сумма дизлайков мема
• сумма лайков паблика, из которого спарсен мем
• сумма дизлайков паблика
• сумма лайков пары юзер-паблик
• сумма дизлайков пары юзер-паблик
• фичи мема: content language, caption language, ...
• фичи юзера: язык, origin, ...
+ инжинирить новые фичи типа общего кол-ва реакций, лайк-рейта (сумма лайков мема на сумму всех реакций)
+ сумма суперлайков, супердизлайков и проч

После чего применяем какой-нибудь gradient boosting из коробки (или любую другую модель из sklearn).

## Нюансы

• Использовать user_id и meme_id как категориальные признаки
• Если тренить модель отдельно на каждого юзера, а у конкретного user_id мало просмотренных мемов, модель может плохо предсказывать
• Целевая метрика precision = TP/P = TP/(TP+FP). Accuracy тут не нужно.
• Имеет смысл сделать отдельную рекомендательную систему для новых юзеров, которые просмотрели мало мемов.

## Тесты

Дается список из n юзеров, надо предсказать m мемов для каждого в формате csv с колонками user_id, meme_id. После чего они просматривают эти мемы и считается средний лайк-рейт, который сравнивается с текущей рекомендательной системой.

In [44]:
event.head(1)

,event_id,reaction_id,session_idx,meme_idx,created_at,updated_at,meme_id,user_id,source_id
0,3607617,2.0,55,11,2020-10-24 19:45:50.964474+00,2020-10-24 19:45:51.049752+00,24379,280747086,45.0


In [45]:
user.head(1)

,user_id,deep_link,is_blocked_bot,is_banned,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender
0,820326025,anime0ch_lang_ru,False,False,False,6,1,20,2020-09-12 20:07:11.668743+00,2020-10-26 12:16:18.458654+00,ru,M


In [46]:
meme.head(1)

,meme_id,meme_type,source_id,published_at,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,content_text,caption_lang,language_id
0,28997,photo,79.0,2020-04-12 01:48:13+00,I mean shitt it ain't their fault,en,True,17,14,2020-04-12 09:04:37.341908+00,2020-10-26 19:20:58.198825+00,False,brandon @brndvx females live on recruit diffic...,en,3.0


Количество просмотренных мемов

In [47]:
event[['reaction_id', 'user_id']].groupby('user_id').count().sort_values('reaction_id', ascending = False).head(15)

,reaction_id
user_id,
81745542,47066
366481144,37491
431589633,36363
420302986,33748
376084050,27982
1254871249,27550
211921528,23956
936739890,23518
248960545,23394


## Анализ англоговорящих сессий

In [48]:
user_en = user.loc[user.interface_lang == 'en']['user_id']
event_en = event[event.user_id.isin(user_en)][['reaction_id', 'user_id']].groupby('user_id').count().sort_values('reaction_id', ascending = False).rename(columns = {'reaction_id':'count'})

In [49]:
event_en.head(5)

,count
user_id,
431589633,36363
941088926,4605
1150038020,3951
544090166,3871
1153477714,3493


In [71]:
print('Среднеквадратичное отклонение: ', event_en.std()[0])
print('Медиана: ', event_en.median()[0])
print('Среднее количество просмотренных мемов: ', event_en.mean()[0])

truncation = len(event_en)//10 #усечение на 10%
print('Среднее усеченное количество просмотренных мемов: ', event_en[truncation:][::-1][truncation:].mean()[0])

Среднеквадратичное отклонение:  517.7767110304451
Медиана:  10.0
Среднее количество просмотренных мемов:  47.53766279479056
Среднее усеченное количество просмотренных мемов:  14.84469863616366


In [53]:
event_en.value_counts().head(15)

count
3        356
4        329
6        314
5        304
7        287
2        260
0        235
8        222
1        218
9        186
11       182
10       170
13       154
12       149
14       125
dtype: int64

## Анализ русскоговорящих сессий

In [54]:
user_ru = user.loc[user.interface_lang == 'ru']['user_id']
event_ru = event[event.user_id.isin(user_ru)][['reaction_id', 'user_id']].groupby('user_id').count().sort_values('reaction_id', ascending = False).rename(columns = {'reaction_id':'count'})

In [55]:
event_ru.head(5)

,count
user_id,
81745542,47066
366481144,37491
420302986,33748
376084050,27982
1254871249,27550


In [69]:
print('Среднеквадратичное отклонение: ', event_ru.std()[0])
print('Медиана: ', event_ru.median()[0])
print('Среднее количество просмотренных мемов: ', event_ru.mean()[0])

truncation = len(event_ru)//10 #усечение на 10%
print('Среднее усеченное количество просмотренных мемов: ', event_ru[truncation:][::-1][truncation:].mean()[0])

Среднеквадратичное отклонение:  1154.2435448537656
Медиана:  28.0
Среднее количество просмотренных мемов:  235.66188334404225
Среднее усеченное количество просмотренных мемов:  64.60096368341215


In [59]:
event_ru.value_counts().head(15)

count
0        434
3        382
4        368
1        330
5        330
8        323
6        319
19       306
20       295
2        291
9        290
14       280
7        261
10       249
15       236
dtype: int64

In [60]:
user

,user_id,deep_link,is_blocked_bot,is_banned,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender
0,820326025,anime0ch_lang_ru,False,False,False,6,1,20,2020-09-12 20:07:11.668743+00,2020-10-26 12:16:18.458654+00,ru,M
1,1040875164,likemeupbot_lang_en,False,False,False,2,1,7,2020-07-21 05:54:12.671036+00,2020-10-26 14:37:05.637591+00,en,NaN
2,449432539,debil_ebaniy,True,False,False,2,1,9,2020-09-14 19:02:06.80911+00,2020-10-17 21:26:52.029824+00,ru,F
3,230248850,NaN,False,False,False,26,1,38,2020-03-16 00:36:42.252438+00,2020-10-26 12:11:02.159405+00,ru,M
4,7719424,NaN,True,False,False,27,6,49,2020-03-15 22:30:45.300242+00,2020-10-17 21:26:52.19941+00,ru,M
...,...,...,...,...,...,...,...,...,...,...,...,...
19913,184147840,NaN,False,False,False,51,67,2564,2020-10-05 06:47:49.485552+00,2020-10-26 19:39:54.897219+00,ru,NaN
19914,663303657,ffmemescom,False,False,False,1,79,79,2020-10-26 19:41:43.90702+00,2020-10-26 19:58:41.257713+00,en,NaN
19915,761418142,popytki_lang_ru,False,False,False,14,21,372,2020-09-30 13:31:29.322251+00,2020-10-26 19:59:29.646093+00,ru,NaN
19916,669142397,ffmemescom,False,False,False,1,33,33,2020-10-26 19:52:20.815557+00,2020-10-26 19:59:39.199266+00,en,NaN


## Выводы

In [61]:
print('EN юзеров',event_en.shape[0],'\nRU юзеров', event_ru.shape[0])

EN юзеров 5682 
RU юзеров 14007


In [62]:
print('Среднее количество просмотренных мемов разнится на', abs(int(event_ru.mean()[0]-event_en.mean()[0])))

Среднее количество просмотренных мемов разнится на 188
